In [80]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MyApp, master=local) created by __init__ at <ipython-input-2-1ab79062268f>:2 

In [81]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + [next_v])

n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x = 12
y = 34
check_y = 0
path = sc.parallelize([(x, [x])]).partitionBy(n)
while True:
  path = path.join(forward_edges, n).map(step)
  check_y = path.filter(lambda i: i[0] == y).count()
  if check_y:
    break
path = path.filter(lambda i:i[0]==y).collect()
print(','.join(map(str,path[0][1])))

12,422,53,52,107,20,23,274,34
